# Team Payroll SQL Analysis: Payroll Per Win (ROI)
## 5-Step Analytics Framework
...

In [1]:
import os
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv

load_dotenv()

pg_user = os.getenv('PG_USER')
pg_password = os.getenv('PG_PASSWORD')
pg_host = os.getenv('PG_HOST')
pg_db = os.getenv('PG_DB')

engine = create_engine(
    f"postgresql+psycopg2://{pg_user}:{pg_password}@{pg_host}:5432/{pg_db}"
)

pd.set_option('display.max_rows', None)

In [2]:
sql_query = '''
WITH payrolls AS (
    SELECT
        "Team Full" AS team,
        "Year" AS year,
        REPLACE(REPLACE("Total Payroll Allocations", '$', ''), ',', '')::numeric AS payroll,
        "Wins_y"::numeric AS wins
    FROM sql_project.team_payroll_records
),
payroll_per_win AS (
    SELECT
        team,
        year,
        payroll,
        wins,
        CASE 
            WHEN wins > 0 THEN payroll / wins 
            ELSE NULL 
        END AS payroll_per_win
    FROM payrolls
)

SELECT *
FROM payroll_per_win
ORDER BY payroll_per_win DESC
LIMIT 20;
'''

df = pd.read_sql(sql_query, engine)
df

,team,year,payroll,wins,payroll_per_win
0,New York Mets,2023,341673777.0,75.0,4.555650e+06
1,New York Mets,2024,314748899.0,89.0,3.536504e+06
2,Boston Red Sox,2020,84210390.0,24.0,3.508766e+06
3,New York Yankees,2023,278651150.0,82.0,3.398185e+06
4,New York Yankees,2020,111939081.0,33.0,3.392093e+06
5,New York Yankees,2024,308574607.0,94.0,3.282709e+06
6,Los Angeles Dodgers,2015,301735080.0,92.0,3.279729e+06
7,Chicago White Sox,2024,132497547.0,41.0,3.231647e+06
8,New York Mets,2020,81945598.0,26.0,3.151754e+06
9,Los Angeles Angels,2023,227884276.0,73.0,3.121702e+06


**Insight:** Some teams spent disproportionately high amounts per win.  
**Recommendation:** Evaluate spending efficiency relative to team success.  
**Prediction:** Higher spending does not guarantee better performance.

In [3]:
sql_query = '''
WITH payrolls AS (
    SELECT
        "Team Full" AS team,
        "Year" AS year,
        REPLACE(REPLACE("Total Payroll Allocations", '$', ''), ',', '')::numeric AS payroll,
        "Wins_y"::numeric AS wins
    FROM sql_project.team_payroll_records
),
payroll_per_win AS (
    SELECT
        team,
        year,
        payroll,
        wins,
        CASE 
            WHEN wins > 0 THEN payroll / wins 
            ELSE NULL 
        END AS payroll_per_win
    FROM payrolls
)

SELECT *
FROM payroll_per_win
WHERE payroll_per_win IS NOT NULL
ORDER BY payroll_per_win ASC
LIMIT 10;
'''

df2 = pd.read_sql(sql_query, engine)
df2

,team,year,payroll,wins,payroll_per_win
0,Tampa Bay Rays,2011,45386925.0,91.0,498757.417582
1,Baltimore Orioles,2022,44888388.0,83.0,540823.951807
2,Houston Astros,2013,28727913.0,51.0,563292.411765
3,Miami Marlins,2013,36209554.0,62.0,584025.064516
4,Cleveland Guardians,2021,50670534.0,80.0,633381.675000
5,Oakland Athletics,2012,60470192.0,94.0,643299.914894
6,Miami Marlins,2014,50559679.0,77.0,656619.207792
7,Pittsburgh Pirates,2011,47976645.0,72.0,666342.291667
8,Tampa Bay Rays,2019,64178722.0,96.0,668528.354167
9,San Diego Padres,2011,48018810.0,71.0,676321.267606


**Insight:** Low-payroll teams occasionally outperform, achieving superior ROI.  
**Recommendation:** Investigate roster management and talent development in these teams.  
**Prediction:** Efficient spending strategies can be replicated by other franchises.